In [ ]:
import os

# List folders/files inside the extracted dataset
for root, dirs, files in os.walk('/kaggle/input/black-faces-data2-3/dataset_2_and_3'):
    print(root, dirs, files)
    break  


/kaggle/input/black-faces-data2-3/dataset_2_and_3 ['dresses', 'upper_body', 'lower_body'] ['train_pairs.txt', 'test_pairs_unpaired.txt', 'test_pairs_paired.txt']


In [ ]:
# Clone the repository
!git clone https://github.com/miccunifi/ladi-vton.git

# Change working directory in the notebook
import os
os.chdir("ladi-vton")

Cloning into 'ladi-vton'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 109 (delta 25), reused 18 (delta 18), pack-reused 75 (from 1)
Receiving objects: 100% (109/109), 1.54 MiB | 9.85 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [3]:
!pip install numpy==1.24.4 torch==2.0.1 torchvision==0.15.2 opencv-python==4.7.0.72 \
    diffusers==0.14.0 transformers==4.27.3 accelerate==0.18.0 clean-fid==0.1.35 \
    "torchmetrics[image]==0.11.4" wandb==0.14.0 matplotlib==3.7.1 tqdm xformers \
    huggingface_hub==0.16.4 jax==0.4.14 jaxlib==0.4.14 flax==0.7.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of optax to determine which version is compatible with other requirements. Thi

In [4]:
# # Download pre-trained weights
# !wget -O /kaggle/working/ladi-vton/checkpoints/warping_dresscode.pth "https://github.com/miccunifi/ladi-vton/releases/download/weights/warping_dresscode.pth"
# !wget -O /kaggle/working/ladi-vton/checkpoints/emasc_dresscode.pth "https://github.com/miccunifi/ladi-vton/releases/download/weights/emasc_dresscode.pth"
# !wget -O /kaggle/working/ladi-vton/checkpoints/unet_dresscode.pth "https://github.com/miccunifi/ladi-vton/releases/download/weights/unet_dresscode.pth"
# !wget -O /kaggle/working/ladi-vton/checkpoints/inversion_adapter_dresscode.pth "https://github.com/miccunifi/ladi-vton/releases/download/weights/inversion_adapter_dresscode.pth"

# **dataset upload** 

In [4]:
!mkdir -p /kaggle/working/ladi-vton/datasets
!cp -r /kaggle/input/black-faces-data2-3/dataset_2_and_3  /kaggle/working/ladi-vton/datasets/

Editing files for debugging

In [ ]:
!cp /kaggle/input/train-files-3000/train_emasc_3000.py /kaggle/working/ladi-vton/src/train_emasc.py
!cp /kaggle/input/train-files-3000/train_tps.py  /kaggle/working/ladi-vton/src/train_tps.py
!cp /kaggle/input/train-files-3000/train_inversion_adapter_3000.py /kaggle/working/ladi-vton/src/train_inversion_adapter.py

done1


updated files 

In [ ]:
import os
os.remove('/kaggle/working/ladi-vton/src/models/ConvNet_TPS.py')
os.remove('/kaggle/working/ladi-vton/src/dataset/dresscode.py')

print("deleted")

deleted


In [ ]:
!cp -r /kaggle/input/updated-files-for-models/ConvNet_TPS.py  /kaggle/working/ladi-vton/src/models
!cp -r /kaggle/input/updated-files-for-models/dresscode.py  /kaggle/working/ladi-vton/src/dataset

print("coppied all required")

coppied all required


In [ ]:
# Create all necessary checkpoint directories
!mkdir -p /kaggle/working/ladi-vton/checkpoints
!mkdir -p /kaggle/working/ladi-vton/checkpoints/WARPING
!mkdir -p /kaggle/working/ladi-vton/checkpoints/EMASC
!mkdir -p /kaggle/working/ladi-vton/checkpoints/INVERSION
!mkdir -p /kaggle/working/ladi-vton/checkpoints/UNET

!cp /kaggle/input/warpingmodel-afterupdate/warping_dresscode/checkpoint_last.pth /kaggle/working/ladi-vton/checkpoints/WARPING/tps.pth
!cp /kaggle/input/new-emasc-7000final/emasc_finetuned.pth /kaggle/working/ladi-vton/checkpoints/EMASC/emasc.pth
!cp /kaggle/input/modelsdataset2and3/inversion_adapter/inversion_adapter.pth  /kaggle/working/ladi-vton/checkpoints/INVERSION/inversion_adapter.pth
!cp /kaggle/input/modelsdataset2and3/unet/unet.pth /kaggle/working/ladi-vton/checkpoints/UNET/unet.pth
print("done")

done


In [18]:
!mkdir -p /kaggle/working/ladi-vton/checkpoints/WARPING/warping_dresscode_optimized
!cp /kaggle/working/ladi-vton/checkpoints/WARPING/tps.pth \
   /kaggle/working/ladi-vton/checkpoints/WARPING/warping_dresscode_optimized/last.pth

****Training wrapping tps****

In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"

!WANDB_DISABLED=true PYTORCH_CUDA_ALLOC_CONF="max_split_size_mb:64" \
python3 /kaggle/working/ladi-vton/src/train_tps.py \
  --dataset dresscode \
  --dresscode_dataroot /kaggle/working/ladi-vton/datasets/dataset_2_and_3 \
  --checkpoints_dir /kaggle/working/ladi-vton/checkpoints/WARPING \
  --exp_name warping_dresscode \
  --epochs_tps 50 \
  --epochs_refinement 50 \
  --lr 1e-4 \
  --batch_size 8 

In [ ]:
import zipfile
# Paths
checkpoint_dir = "/kaggle/working/ladi-vton/checkpoints/WARPING/warping_dresscode"
output_zip = "/kaggle/working/warping_checkpoint.zip"

# Create ZIP
with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(checkpoint_dir):
        for file in files:
            if file.endswith('.pth'):  
                zipf.write(
                    os.path.join(root, file), 
                    os.path.relpath(os.path.join(root, file), checkpoint_dir)
                )
print(f"Checkpoint zipped to: {output_zip}")

Checkpoint zipped to: /kaggle/working/warping_checkpoint.zip


**emasc**

In [ ]:
#emasc
import os
os.environ["WANDB_MODE"] = "disabled"

!WANDB_DISABLED=true PYTORCH_CUDA_ALLOC_CONF="max_split_size_mb:64"  \
python /kaggle/working/ladi-vton/src/train_emasc.py \
    --dataset dresscode \
    --dresscode_dataroot /kaggle/working/ladi-vton/datasets/dataset_2_and_3 \
    --output_dir /kaggle/working/ladi-vton/checkpoints/emasc \
    --train_batch_size 1 \
    --mixed_precision fp16 \
    --num_workers 2 \
    --num_train_epochs 10 \
    --max_train_steps 7000 \
    --test_order paired

2025-06-15 11:30:50.645163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-15 11:30:50.668924: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-15 11:30:50.676118: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
{'scaling_factor'} was not found in config. Values will be initialized to default values.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please 

In [ ]:
folder_path = '/kaggle/working/ladi-vton/checkpoints' 

for dirpath, dirnames, filenames in os.walk(folder_path):
    print(f"📁 Folder: {dirpath}")
    for dirname in dirnames:
        print(f"    📂 Subfolder: {dirname}")
    for filename in filenames:
        print(f"    📄 File: {filename}")

**inference**

In [21]:
# clip

import os
os.environ["WANDB_MODE"] = "disabled"

!python src/utils/compute_cloth_clip_features.py \
    --dataset dresscode \
    --dresscode_dataroot /kaggle/working/ladi-vton/datasets/dataset_2_and_3 \
    --pretrained_model_name_or_path stabilityai/stable-diffusion-2-inpainting \
    --batch_size 16 
# --num_workers 4

2025-06-15 05:33:54.729566: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-15 05:33:55.386618: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-15 05:33:55.568965: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Some weights of the model checkpoint at laion/CLIP-ViT-H-14-laion2B-s32B-b79K were not used when initializing CLIPVisionModelWithProjection: ['text_model.encoder.layers.17.layer_norm2.weight', 'text_model.encoder.layers.15.self_attn.k_proj.bias', 'text_model.encoder.layers.22.layer_norm2.bias', 'text_model.encoder.layers.0.self_attn.k_proj.weight', 'text_model.en

In [ ]:
!python src/eval.py \
  --dataset dresscode \
  --dresscode_dataroot /kaggle/working/ladi-vton/datasets/dataset_2_and_3 \
  --output_dir /kaggle/working/results \
  --save_name my_tryon_output \
  --test_order unpaired \
  --unet_dir /kaggle/working/ladi-vton/checkpoints/UNET \
  --unet_name unet.pth \
  --emasc_dir /kaggle/working/ladi-vton/checkpoints/emasc \
  --emasc_name emasc_finetuned.pth \
  --inversion_adapter_dir /kaggle/working/ladi-vton/checkpoints/INVERSION \
  --inversion_adapter_name inversion_adapter.pth \
  --text_usage inversion_adapter \
  --cloth_input_type warped \
  --category all \
  --enable_xformers_memory_efficient_attention \
  --num_workers 8 \
  --batch_size 1 \
  --num_inference_steps 60 \
  --compute_metrics \
  --use_clip_cloth_features

In [ ]:
!python /kaggle/working/ladi-vton/src/inference.py \
  --output_dir /kaggle/working/inference_results \
  --mixed_precision fp16 \
  --dataset dresscode \
  --dresscode_dataroot /kaggle/working/ladi-vton/datasets/dataset_2_and_3  \
  --test_order unpaired \
  --category all \
  --num_vstar 16  \
  --num_inference_steps 60  \
  --guidance_scale 7.5 \
  --compute_metrics

2025-06-28 12:06:52.671456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-28 12:06:52.696177: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-28 12:06:52.703304: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Some weights of the model checkpoint at laion/CLIP-ViT-H-14-laion2B-s32B-b79K were not used when initializing CLIPVisionModelWithProjection: ['text_model.encoder.layers.5.self_attn.k_proj.weight', 'text_model.encoder.layers.9.self_attn.k_proj.bias', 'text_model.encoder.layers.10.layer_norm2.bias', 'text_model.encoder.layers.20.self_attn.q_proj.weight', 'text_mode

In [ ]:
import shutil
# Zipping inference results
output_dir = "/kaggle/working/inference_results"
shutil.make_archive(output_dir, 'zip', output_dir)

print("📦results zipped to:", output_dir + ".zip")


In [ ]:

import shutil
# Zipping eval results
output_dir = "/kaggle/working/ladi-vton/checkpoints/emasc"
shutil.make_archive(output_dir, 'zip', output_dir)

print("📦 new checkpoints emasc:", output_dir + "emasc.zip")
